Restrições hard:
Não podemos alocar uma turma numa sala com quantidade de assentos menor
que o número de alunos matriculados.
Não podemos alocar uma turma que necessita de acessibilidade numa sala que
não é acessível
Restrições soft
• Devemos minimizar a taxa de ocupação de salas: (número de alunos
– tamanho da sala)/tamanho da sala.
• Restrição de qualidade da sala: Uma turma de qualidade baixa pode
ser alocada numa sala de qualidade alta, mas o contrário não é
verdade.
• Restrição de salas no mesmo horário: penalizar uma distribuição
onde uma mesma turma esteja numa sala diferentes.
• Restrição de calouros na mesma sala: penalizar distribuições onde
turmas de calouros de um mesmo curso estejam em salas
diferentes.

In [7]:
#definitions

import csv
#leitura andre
#df = csv.DictReader(open("cenario1-salas.csv")) 
#df1 = csv.DictReader(open("cenario1-turmas.csv",encoding='utf-8'))

#leitura paulo
df = csv.DictReader(open(r"C:\Users\Paulo\Desktop\TrabalhoSAD\cenario1-salas.csv",encoding='utf-8'))
df1 = csv.DictReader(open(r"C:\Users\Paulo\Desktop\TrabalhoSAD\cenario1-turmas.csv",encoding='utf-8'))
turmas = {}
salas = {}
horarios = {}
id_Salas = []
horario = {}
x = 0

In [8]:
#transforma o horario para um formato mais amigavel
def transHorario(horario):
    list = []
    print(horario)
    turmaTemp = horario.split("-")
    for i in range(len(turmaTemp)):
        dia = int(turmaTemp[i][0])
        turno = turmaTemp[i][1]
        horarioIni = int(turmaTemp[i][2])

        if(turno == 'M'):
            horarioIni = 1 + horarioIni/2 
        if(turno == 'T'):
            horarioIni = 4 + horarioIni/2 

        elif(turno == 'N'):
            horarioIni = 7 + horarioIni/2
        
        if(int(turmaTemp[i][0]) == 2):
            dia = 'seg'
        elif(int(turmaTemp[i][0]) == 3):
            dia = 'ter'
        elif(int(turmaTemp[i][0]) == 4):
            dia = 'qua'
        elif(int(turmaTemp[i][0]) == 5):
            dia = 'qui'
        elif(int(turmaTemp[i][0]) == 6):
            dia = 'sex'
        else:
            dia = 'sab'
        list.append((dia,int(horarioIni)))
    print(list)
    return list

In [12]:
    
for row in df1:
    turmas[x] = {"disciplina": row['disciplina'],"prof": row['professor'], "horario": transHorario(row['dias_horario']),
                 "alunos": int(row['numero_alunos']), "curso": row['curso'], 
                 "periodo": int(row['período']), "acess": int(row["acessibilidade"]), "quali": int(row["qualidade"])}
    x+=1
for row in df:
    salas[row['id_sala']] = {"cad": int(row['numero_cadeiras']), "acess": int(row['acessivel']), "quali": row['qualidade']}
    id_Salas.append(row['id_sala'])

for sala in id_Salas:
    horarios[sala] =  { "seg":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "ter":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "qua":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "qui":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "sex":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "sab":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 }}




In [13]:

sorted_salas = sorted(salas.items(), key=lambda sala: (sala[1]['acess'],sala[1]['cad']), reverse=True)

sorted_turmas = sorted(turmas.items(), key=lambda turma: (turma[1]['acess'], turma[1]['alunos']), reverse=True)


In [14]:
def achaSalasDisponiveis(horarios, turma, salas):
    h = turma['horario']
    #recebe a lista de horarios dessa turma
    salasDisponiveis = []
    #lista que recebe a solução
    for sala in salas:
        
        for j in range(len(horarios)):
            contador = 0
            for k in range(len(h)):
                dia = h[k][0]
                #recebe o dia do horario k 
                horario = h[k][1] 
                #recebe a hora do horario k
                if(horarios[sala[0]][dia][horario] == 0): 
                    contador += 1 
                    #se o horario estiver disponivel é incrementado este contador
        if(contador == len(h)): 
            #se o contador chegar ao mesmo tamanho da lista de horario, significa que essa 
            # sala está livre em todos os horarios dessa turma, logo ela pode ser alocada.
            salasDisponiveis.append(sala) #adiciona a sala avaliada na lista resultado
    return salasDisponiveis

In [16]:
def achaSalasPossiveis(horarios, turma, salas):
    salasDisponiveis = achaSalasDisponiveis(horarios,turma, salas) 
    #busca as salas que ainda estão livres
    salasPossiveis = []
    for sala in salasDisponiveis:
        # Verifica a acessibilidade e o tamanho da turma. Ex:
        # Acess = 0 retornar turmas com acess 0 e 1
        # Acess = 1 retornar turmas com acess 1
        if(sala[1]['cad'] >= turma['alunos'] and sala[1]['acess'] >= turma['acess']): 
            salasPossiveis.append(sala)
    return salasPossiveis

In [269]:
def alocaTurmas(horarios, turmas, salas):
    salasPossiveis = []
    for turma in turmas:
        h = turma[1]['horario']
        salasPossiveis = achaSalasPossiveis(horarios, turma[1], salas)
        if(salasPossiveis == []):
            print("deu conflito", turma[1])
        else:
            for i in range(len(h)):
                dia = h[i][0] 
                #recebe o dia do horario k 
                horario = h[i][1] 
                #recebe a hora do horario k
                sala = salasPossiveis[0][0]
                horarios[sala][dia][horario] = 1
                #print("aloquei", salasPossiveis[0], horarios[salasPossiveis[0]][dia][horario] )
    return horarios
            
x = alocaTurmas(horarios, sorted_turmas, sorted_salas)

for sala in sorted_salas:
    print(x[sala[0]])
#achaSalasPossiveis(horarios, turmas[0], salas)

{'seg': {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1}, 'ter': {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1}, 'qua': {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0, 7: 1, 8: 1}, 'qui': {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1}, 'sex': {1: 0, 2: 1, 3: 0, 4: 1, 5: 1, 6: 1, 7: 0, 8: 1}, 'sab': {1: 1, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0}}
{'seg': {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1}, 'ter': {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1}, 'qua': {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1}, 'qui': {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1}, 'sex': {1: 1, 2: 0, 3: 1, 4: 1, 5: 1, 6: 0, 7: 1, 8: 0}, 'sab': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0}}
{'seg': {1: 0, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0}, 'ter': {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0}, 'qua': {1: 0, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0}, 'qui': {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0}, 'sex': {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 1, 7: 0, 8: 0}, 'sab': {1: 0, 2: 0, 3: 0, 4: